In [1]:
# Install Ngrok
!pip install pyngrok flask flask-ngrok
# install Hugging Face Libraries
!pip install "peft==0.2.0"
!pip install "transformers==4.27.2" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [2]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes

Found existing installation: bitsandbytes 0.37.1
Uninstalling bitsandbytes-0.37.1:
  Successfully uninstalled bitsandbytes-0.37.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok
from google.colab import drive
import os
from pyngrok import ngrok


In [4]:
# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
model_id="google/flan-t5-xxl"

# Load tokenizer of FLAN-t5-XXL
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
# huggingface hub model id
model_id = "philschmid/flan-t5-xxl-sharded-fp16"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00012.bin:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

pytorch_model-00002-of-00012.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

...

pytorch_model-00009-of-00012.bin:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

pytorch_model-00010-of-00012.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00011-of-00012.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00012-of-00012.bin:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [7]:
adapter_model_path = "LoRA"

# Load the LoRA adapter
model = PeftModel.from_pretrained(model, adapter_model_path)


In [12]:
import re
from flask import jsonify, request
import torch

In [17]:
def clean_text(text):
    # Remove page numbers (assuming they're at the end of lines)
    text = re.sub(r'\s+\d+\s*$', '', text, flags=re.MULTILINE)

    # Remove section numbers
    text = re.sub(r'^\s*\d+(\.\d+)*\s*', '', text, flags=re.MULTILINE)

    # Remove dots between words and numbers
    text = re.sub(r'\s*\.\s*', ' ', text)

    # Remove any remaining numbers
    text = re.sub(r'\d+', '', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove leading and trailing whitespace
    text = text.strip()

    # Split the text into lines, remove empty lines, and join back
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    text = '\n'.join(lines)




    return text

In [18]:
# Set your Ngrok auth token
NGROK_AUTH_TOKEN = "token"  # Update this with your Ngrok auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return render_template_string(open('index.html').read())

@app.route('/summarize', methods=['POST'])
def summarize():
  text = request.json['text']
  cleaned_text = clean_text(text)
  input_ids = tokenizer("summrize: " + cleaned_text, return_tensors="pt", truncation=True).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=350, min_length=50, do_sample=True, top_p=0.9)
  summary = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return jsonify({'summary': summary})



@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files['file']
    if file and file.filename.endswith('.txt'):
        text = file.read().decode('utf-8')
        return jsonify({'text': text})
    else:
        return jsonify({'error': 'Unsupported file type. Please upload a .txt file.'}), 400

# Run the Flask app
if __name__ == "__main__":
    # Setup ngrok

    # Open a ngrok tunnel to the public internet
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))

    app.run()

 * ngrok tunnel "NgrokTunnel: "https://08db-34-125-128-110.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://08db-34-125-128-110.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:51:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:51:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:52:27] "POST /summarize HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:52:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:55:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:55:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:56:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:58:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:58:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 09:59:13] "POST /summarize HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 10:07:26] "POST /summarize HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 10:08:18] "POST /summarize HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2024 10:09:15] "POST /sum